In [ ]:
# 隐匿查询
#
# 配置描述     |基础运算            |联合统计       |隐匿查询          |安全求交        |特征工程           |联合建模   |联合预测 
# 数据方       |两方, 三方          |两方          |两方              |两方            |两方              |两方       |两方 
# 算法类型     |加法, 乘法, 比较    |方差, 中位数   |高效率, 高隐匿性   |平衡, 非平衡    |WOE和IV计算        |LR        |LR
# 测试数据集   |数据集α             |数据集α       |数据集β            |数据集γ        |epsilon           |epsilon    |epsilon

import os
path = './ds'
assert os.path.exists(path)

In [ ]:
%%time

import pyarrow as pa 
import pyarrow.dataset as ds
import pyarrow.compute as pc

beta_ds = ds.dataset(f'{path}/ds_beta/')
proj = {col:ds.field(col) for col in beta_ds.schema.names}
# proj.update({'ID': ds.field('ID').cast(pa.string())})
beta = beta_ds.to_table(columns=proj)
# beta = beta.sort_by('ID')

In [ ]:
%%time

query_ds = ds.dataset(f'{path}/ds_beta_query/')
proj = {col:ds.field(col) for col in query_ds.schema.names}
# proj.update({'ID': ds.field('ID').cast(pa.string())})
query = query_ds.to_table(columns=proj)

In [ ]:
%%time

idx = pc.index_in(query['ID'], value_set=beta['ID'])
result = beta.take(idx)

In [ ]:
%%time

import modin.config as cfg
cfg.Engine.put("Ray")
cfg.CpuCount.put("4")
#cfg.StorageFormat.put('Pyarrow')   # read_parquet doesn't support PyarrowOnRay yet
cfg.StorageFormat.put('Pandas')

# or by setting the environment variable
# import os
# os.environ["MODIN_ENGINE"] = "ray"
# os.environ["MODIN_STORAGE_FORMAT"] = "Pyarrow"

# import pandas # Pandas couldn't fit our large dataset into memory
import modin.pandas as pd
#import modin.experimental.pandas as pd
import pyarrow as pa 
import pyarrow.parquet as pq
import numpy as np

import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

df = pd.read_parquet(f'{path}/ds_beta/')
df = df.reindex(['ID'])
df.sort_index(inplace=True)
df_query = pd.read_parquet(f'{path}/ds_beta_query/')
df_rs = df[df_query['ID']]